In [1]:
import pickle
import pandas as pd

In [2]:
with open("../data/interim/01_article_comparison_info.pkl", "rb") as f:
    raw_df = pickle.load(f)

In [4]:
raw_df.head(2)

,parent_title,parent_url,child_title,child_url,comparison,parent_extract,parent_links,parent_linkshere,parent_categories,parent_redirects,child_extract,child_links,child_linkshere,child_categories,child_redirects
0,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_Zero_(story)|Division_by_zero,Division by Zero is a science fiction short st...,"[Amazon.com, Anthology, Bantam Spectra, Consis...","[Division by zero (disambiguation), Stories of...","[Category:1991 short stories, Category:Science...",[],"In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,..."
1,Indeterminate_form,https://en.wikipedia.org/wiki/Indeterminate_form,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_zero|Indeterminate_form,In calculus and other branches of mathematical...,"[Analytic function, Calculus, Cardinal number,...","[Exponentiation, Indeterminate expression, L'H...",[Category:Limits (mathematics)],"[0/0, Zero divided by zero, 0 divided by 0, In...","In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,..."


## Helper Matching Functions

In [12]:
def list_match_bool(lst1, lst2):
    for item in lst1:
        if item in lst2:
            return True
    return False

def list_match_count(lst1, lst2):
    matches = 0
    for item in lst1:
        if item in lst2:
            matches += 1
            
    return matches

def match_str_in_list(string, lst):
    return True if string.replace("_", " ") in lst else False

### parent_direct_to_child:

> Boolean - if any of the parent_links match the child_title or any links in the child_redirects

In [13]:
def parent_to_child(row):
    return match_str_in_list(row.child_title, row.parent_links)

raw_df['parent_direct_to_child'] = raw_df.apply(parent_to_child, axis=1)

### child_direct_to_parent:

> Boolean - if any of the child_links match the parent_title or any links in the parent_redirects

In [15]:
def child_to_parent(row):
    return match_str_in_list(row.parent_title, row.child_links)

raw_df['child_direct_to_parent'] = raw_df.apply(child_to_parent, axis=1)

### parent_path_to_child:

> number of parent_links that are found also in the child_linkshere

In [17]:
def path_to_child(row):
    return list_match_count(row.parent_links, row.child_linkshere)

raw_df["parent_path_to_child"] = raw_df.apply(path_to_child, axis=1)

### child_path_to_parent:

> number of child_links that are found also in the parent_linkshere

In [19]:
def path_to_parent(row):
    return list_match_count(row.child_links, row.parent_linkshere)

raw_df["child_path_to_parent"] = raw_df.apply(path_to_parent, axis=1)

### shared_categories:

> number of categories shared between the parent_categories and child_categories

In [21]:
def category_matches(row):
    return list_match_count(row.parent_categories, row.child_categories)

raw_df["shared_categories"] = raw_df.apply(category_matches, axis=1)

# Preview and Save

In [23]:
raw_df.head(10)

,parent_title,parent_url,child_title,child_url,comparison,parent_extract,parent_links,parent_linkshere,parent_categories,parent_redirects,child_extract,child_links,child_linkshere,child_categories,child_redirects,parent_direct_to_child,child_direct_to_parent,parent_path_to_child,child_path_to_parent,shared_categories
0,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_Zero_(story)|Division_by_zero,Division by Zero is a science fiction short st...,"[Amazon.com, Anthology, Bantam Spectra, Consis...","[Division by zero (disambiguation), Stories of...","[Category:1991 short stories, Category:Science...",[],"In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,...",True,True,1,2,0
1,Indeterminate_form,https://en.wikipedia.org/wiki/Indeterminate_form,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_zero|Indeterminate_form,In calculus and other branches of mathematical...,"[Analytic function, Calculus, Cardinal number,...","[Exponentiation, Indeterminate expression, L'H...",[Category:Limits (mathematics)],"[0/0, Zero divided by zero, 0 divided by 0, In...","In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,...",True,True,6,9,0
2,0,https://en.wikipedia.org/wiki/0,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,0|Division_by_zero,0 (zero) is both a number and the numerical di...,"[-1 (number), 0 (disambiguation), 0 (number), ...","[7000 (number), 19 (number), 178 (number), EBC...","[Category:0 (number), Category:Elementary arit...","[Number 0, Nought, Zeroes, Zero (mathematics),...","In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,...",True,False,8,2,1
3,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Infinity,https://en.wikipedia.org/wiki/Infinity,Division_by_Zero_(story)|Infinity,Division by Zero is a science fiction short st...,"[Amazon.com, Anthology, Bantam Spectra, Consis...","[Division by zero (disambiguation), Stories of...","[Category:1991 short stories, Category:Science...",[],Infinity (symbol: \n \n \n \n ...,"[0.999..., 1,000,000, 1,000,000,000, 10,000, 1...","[Iterated binary operation, Bill Childers, Chr...","[Category:CS1: long volume value, Category:Con...","[The Infinite, Infinitely, Infinate, InFINity,...",True,False,2,1,0
4,Indeterminate_form,https://en.wikipedia.org/wiki/Indeterminate_form,Extended_real_number_line,https://en.wikipedia.org/wiki/Extended_real_nu...,Extended_real_number_line|Indeterminate_form,In calculus and other branches of mathematical...,"[Analytic function, Calculus, Cardinal number,...","[Exponentiation, Indeterminate expression, L'H...",[Category:Limits (mathematics)],"[0/0, Zero divided by zero, 0 divided by 0, In...","In mathematics, the affinely extended real num...","[Absolute value, Algebraic number, Arithmetica...","[Number line, L'Hôpital's rule, 1,000,000,000,...","[Category:Infinity, Category:Real numbers]","[Extended real number, Extended reals, Extende...",True,True,8,6,0
5,Number_line,https://en.wikipedia.org/wiki/Number_line,Extended_real_number_line,https://en.wikipedia.org/wiki/Extended_real_nu...,Extended_real_number_line|Number_line,"In basic mathematics, a number line is a pictu...","[Addition, Atom, Brooks Cole, Cartesian coordi...","[Coordinate system, Cartesian coordinate syste...","[Category:Elementary ma

In [24]:
with open("../data/interim/02_extracted_basic_numerical_comparison.pkl", "wb") as f:
    pickle.dump(raw_df, f, 3)